<a href="https://colab.research.google.com/github/Sakuni-Weerasinghe/Automatic-Question-and-Answer-Generation-based-on-Large-Language-Models/blob/master/MCQGeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install tokenizers
!pip install sentencepiece
!pip install pytorch-lightning
!pip install sense2vec

Import Libraries

In [ ]:
from typing import List, Dict
import tqdm.notebook as tq
from tqdm.notebook import tqdm
import json
import pandas as pd
import numpy as np
import random

from transformers import T5Tokenizer
import seaborn as sns
import matplotlib.pyplot as plt
import pytorch_lightning as pl
from sklearn.model_selection import train_test_split
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import Trainer
from torch.utils.data import Dataset, DataLoader
import torch
from torch.utils.data import Dataset
from typing import Tuple
from typing import List
from nltk.tokenize import sent_tokenize
import toolz
import time
# from platform import dist
import string
import re
import textwrap
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
from sense2vec import Sense2Vec
from collections import OrderedDict
from typing import List


from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5TokenizerFast as T5Tokenizer
    )

##Modules

###Duplicate Romveal

In [ ]:
def remove_duplicates(items: List[str]) -> List[str]:
    unique_items = []
    normalized_unique_items = []

    for item in items:
        normalized_item = _normalize_item(item)

        if normalized_item not in normalized_unique_items:
            unique_items.append(item)
            normalized_unique_items.append(normalized_item)

    return unique_items

def remove_distractors_duplicate_with_correct_answer(correct: str, distractors: List[str]) -> List[str]:
    for distractor in distractors:
        if _normalize_item(correct) == _normalize_item(distractor):
            distractors.remove(distractor)

    return distractors

def _get_most_distinct_from_key(key: str, items: List[str]) -> List[str]:
    #TODO: This seems not as useful. For example "the family Phascolarctidae" and "the family Vombatidae" are close, but good distractors.

    return items

def _get_most_distinct_from_each_other():
    #TODO
    # calculate bleu for each with each.
    # find the most similar pair
    # remove the second in the original list (assuming the list comes ordered by better)
    # run until you get the desired amount
    pass

def _normalize_item(item) -> str:
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(item))))

def _calculate_nltk_bleu(references: List[str], hypothesis: str, bleu_n: int = 1):
    if hypothesis == '':
        return 0, 0, 0, 0

    # Word tokenize
    refs_tokenized = list(map(lambda x: word_tokenize(x), references))
    hyp_tokenized = word_tokenize(hypothesis)

    # Smoothing function to avoid the cases where it resuts 1.0 in the cases when // Corpus/Sentence contains 0 counts of 2-gram overlaps. BLEU scores might be undesirable; use SmoothingFunction() //
    chencherry = SmoothingFunction()
    bleu = 0

    if bleu_n == 1:
        bleu = sentence_bleu(refs_tokenized, hyp_tokenized, weights=(1, 0, 0, 0), smoothing_function=chencherry.method2)
    elif bleu_n == 2:
        bleu = sentence_bleu(refs_tokenized, hyp_tokenized, weights=(0.5, 0.5, 0, 0), smoothing_function=chencherry.method2)
    elif bleu_n == 3:
        bleu = sentence_bleu(refs_tokenized, hyp_tokenized, weights=(0.33, 0.33, 0.33, 0), smoothing_function=chencherry.method2)
    elif bleu_n == 4:
        bleu = sentence_bleu(refs_tokenized, hyp_tokenized, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=chencherry.method2)

    return bleu

###Text Cleaning

In [ ]:
def clean_text(text: str) -> str:
    """Clean the text from symbols and additional information.

    Args:
        text (str): The text.

    Returns:
        str: CLeaned text.
    """
    cleaned_text = _remove_brackets(text)
    cleaned_text = _remove_square_brackets(cleaned_text)
    cleaned_text = _remove_multiple_spaces(cleaned_text)
    cleaned_text = _replace_weird_hyphen(cleaned_text)

    return cleaned_text


def _remove_brackets(text: str) -> str:
    """ Remove brackets '(', ')' and the information between them.

    e.g. "The koala has a body length of 60–85 cm (24–33 in)."

    Args:
        text (str): The text.

    Returns:
        str: CLeaned text.
    """
    return re.sub(r'\((.*?)\)', lambda L: '', text)


def _remove_square_brackets(text: str) -> str:
    """ Remove square brackets '[', ']' and the information between them.

    e.g. The koala[1] is cool."

    Args:
        text (str): The text.

    Returns:
        str: CLeaned text.
    """

    return re.sub(r'\[(.*?)\]', lambda L: '', text)


def _remove_multiple_spaces(text: str) -> str:
    """Remove multiple white spaces.

    e.g. "The koala         is     angry  !"

    Args:
        text (str): The text.

    Returns:
        str: CLeaned text.
    """

    return re.sub(' +', ' ', text)


def _replace_weird_hyphen(text: str) -> str:
    """ Replace weird '–' hyphen that's not recognized as a delimeter by spacy.

    e.g. '4–15 kg' -> '4-15 kg'
    (You may not see a difference, but there fucking is. This motherfucker '–' is not recognized by spacy as a delimeter.)

    Args:
        text (str): The text.

    Returns:
        str: CLeaned text.
    """
    return text.replace('–', '-')

Download Datasets

In [ ]:
#https://drive.google.com/file/d/1ggZqb_jCmq12gwmfBBQhOWUV4dN00S-o/view?usp=sharing
#https://drive.google.com/file/d/1VrBghMWob_-mTz0sB5B8KLkq0MgdRrC9/view?usp=sharing
#https://drive.google.com/file/d/192ibBUshejWnZm3vepWNMi-9kn5GypiD/view?usp=sharing

!gdown --id 1ggZqb_jCmq12gwmfBBQhOWUV4dN00S-o #train.csv
!gdown --id 1VrBghMWob_-mTz0sB5B8KLkq0MgdRrC9 #test.csv
!gdown --id 192ibBUshejWnZm3vepWNMi-9kn5GypiD #val.csv

In [ ]:
train_dataset = pd.read_csv('train.csv')
test_dataset = pd.read_csv('test.csv')
val_dataset = pd.read_csv('val.csv')

In [ ]:
print(train_dataset.shape,'train_dataset')
print(test_dataset.shape, 'test_dataset')
print(val_dataset.shape, 'val_dataset')

(606, 9) train_dataset
(190, 9) test_dataset
(152, 9) val_dataset


In [ ]:
train_dataset_1 = train_dataset.copy()
test_dataset_1 = test_dataset.copy()
val_dataset_1 = val_dataset.copy()

train_dataset_1 = train_dataset_1.dropna() #removing rows with missing values in the DataFrame
test_dataset_1 = test_dataset_1.dropna() #removing rows with missing values in the DataFrame
val_dataset_1 = val_dataset_1.dropna() #removing rows with missing values in the DataFrame

# train_dataset_1.drop(columns=['option1', 'option2','option3','option4'], inplace=True) #answer_start and answer_end are not needed and are for the paragraph
# test_dataset_1.drop(columns=['option1', 'option2','option3','option4'], inplace=True)
# val_dataset_1.drop(columns=['option1', 'option2','option3','option4'], inplace=True)

print(train_dataset_1.shape,'train_dataset_1')
print(test_dataset_1.shape, 'test_dataset_1')
print(val_dataset_1.shape, 'val_dataset_1')


(586, 9) train_dataset_1
(185, 9) test_dataset_1
(150, 9) val_dataset_1


In [ ]:
# Constants
MODEL_NAME = 't5-base'
LEARNING_RATE = 0.0001
SOURCE_MAX_TOKEN_LEN = 200
TARGET_MAX_TOKEN_LEN = 80
SEP_TOKEN = '<sep>'
TOKENIZER_LEN = 32101  # after adding the new <sep> token


In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


# Question Generation Model

In [ ]:
class QGModel(pl.LightningModule):
    def __init__(self):
        super().__init__()

        # Load tokenizer and model
        self.tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
        self.tokenizer.add_tokens([SEP_TOKEN])

        self.model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
        self.model.resize_token_embeddings(TOKENIZER_LEN)

        # Resize the token embeddings to match the new tokenizer length
        self.model.config.max_length = SOURCE_MAX_TOKEN_LEN
        self.model.config.max_length_for_sum = TARGET_MAX_TOKEN_LEN

        # Set learning rate
        self.optimizer = torch.optim.AdamW(self.model.parameters(), lr=LEARNING_RATE)

    def forward(self, input_ids, attention_mask, labels=None):
        output = self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        return output.loss, output.logits

    def training_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        loss, output = self(input_ids, attention_mask, labels)
        self.log('train_loss', loss, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        loss, output = self(input_ids, attention_mask, labels)
        self.log('val_loss', loss, prog_bar=True, logger=True)
        return loss

    def test_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']
        loss, output = self(input_ids, attention_mask, labels)
        self.log('test_loss', loss, prog_bar=True, logger=True)
        return loss

    def configure_optimizers(self):
        return self.optimizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
#https://drive.google.com/file/d/1Vr1dapnNme4IqSJe6cOafutFSe3kBxqU/view?usp=sharing
!gdown --id 1Vr1dapnNme4IqSJe6cOafutFSe3kBxqU

In [ ]:
class QuestionGenerator():
    def __init__(self):
        self.tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
        # print('tokenizer len before: ', len(self.tokenizer))
        self.tokenizer.add_tokens(SEP_TOKEN)
        # print('tokenizer len after: ', len(self.tokenizer))
        self.tokenizer_len = len(self.tokenizer)

        checkpoint_path = 'best-checkpoint-v7.ckpt'
        self.qg_model = QGModel.load_from_checkpoint(checkpoint_path)
        self.qg_model.freeze()
        self.qg_model.eval()

    def generate(self, answer: str, context: str) -> str:
        model_output = self._model_predict(answer, context)

        generated_answer, generated_question = model_output.split('<sep>')

        return generated_question

    def generate_qna(self, context: str) -> Tuple[str, str]:
        answer_mask = '[MASK]'
        model_output = self._model_predict(answer_mask, context)

        qna_pair = model_output.split('<sep>')

        if len(qna_pair) < 2:
            generated_answer = ''
            generated_question = qna_pair[0]
        else:
            generated_answer = qna_pair[0]
            generated_question = qna_pair[1]

        return generated_answer, generated_question

    def _model_predict(self, answer: str, context: str) -> str:
        source_encoding = self.tokenizer(
            '{} {} {}'.format(answer, SEP_TOKEN, context),
            max_length=SOURCE_MAX_TOKEN_LEN,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors='pt'
        ).to('cuda')

        generated_ids = self.qg_model.model.generate(
            input_ids=source_encoding['input_ids'],
            attention_mask=source_encoding['attention_mask'],
            num_beams=16,
            max_length=TARGET_MAX_TOKEN_LEN,
            repetition_penalty=2.5,
            length_penalty=1.0,
            early_stopping=True,
            use_cache=True
        )

        preds = {
            self.tokenizer.decode(generated_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
            for generated_id in generated_ids
        }

        return ''.join(preds)

#Answer Generation

In [ ]:

class AnswerGenerator():
    def __init__(self):
        self.tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)

        checkpoint_path = 'best-checkpoint-v7.ckpt'
        self.ag_model = QGModel.load_from_checkpoint(checkpoint_path)
        self.ag_model.freeze()
        self.ag_model.eval()

    def generate(self, context: str, generate_count: int) -> List[str]:
        model_output = self._model_predict(context, generate_count)

        answers = model_output.replace('<pad>', '').split('</s>')[:-1]

        return answers

    def _model_predict(self, context: str, generate_count: int) -> str:
        source_encoding = self.tokenizer(
            context,
            max_length=SOURCE_MAX_TOKEN_LEN,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors='pt'
        ).to('cuda')

        generated_ids = self.ag_model.model.generate(
            input_ids=source_encoding['input_ids'],
            attention_mask=source_encoding['attention_mask'],
            num_beams=generate_count,
            num_return_sequences=generate_count,
            max_length=TARGET_MAX_TOKEN_LEN,
            repetition_penalty=2.5,
            length_penalty=1.0,
            early_stopping=True,
            use_cache=True
        )

        preds = {
            self.tokenizer.decode(generated_id, skip_special_tokens=False, clean_up_tokenization_spaces=True)
            for generated_id in generated_ids
        }

        return ''.join(preds)

#Distractor Generation

In [ ]:
#https://drive.google.com/file/d/186jLSvapqLPtKsTrPKKFGwfQfd2N38C_/view?usp=sharing

!gdown --id 186jLSvapqLPtKsTrPKKFGwfQfd2N38C_

In [ ]:
class DistractorGenerator():
    def __init__(self):
        self.tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
        # print('tokenizer len before: ', len(self.tokenizer))
        self.tokenizer.add_tokens(SEP_TOKEN)
        # print('tokenizer len after: ', len(self.tokenizer))
        self.tokenizer_len = len(self.tokenizer)

        checkpoint_path = 'best-checkpoint-v10.ckpt'
        self.dg_model = QGModel.load_from_checkpoint(checkpoint_path)
        self.dg_model.freeze()
        self.dg_model.eval()

    def generate(self, generate_count: int, correct: str, question: str, context: str) -> List[str]:

        generate_triples_count = int(generate_count / 3) + 1 #since this model generates 3 distractors per generation

        model_output = self._model_predict(generate_triples_count, correct, question, context)

        cleaned_result = model_output.replace('<pad>', '').replace('</s>', '<sep>')
        cleaned_result = self._replace_all_extra_id(cleaned_result)
        distractors = cleaned_result.split('<sep>')[:-1]
        distractors = [x.translate(str.maketrans('', '', string.punctuation)) for x in distractors]
        distractors = list(map(lambda x: x.strip(), distractors))

        return distractors

    def _model_predict(self, generate_count: int, correct: str, question: str, context: str) -> str:
        source_encoding = self.tokenizer(
            '{} {} {} {} {}'.format(correct, SEP_TOKEN, question, SEP_TOKEN, context),
            max_length= SOURCE_MAX_TOKEN_LEN,
            padding='max_length',
            truncation= True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors='pt'
            ).to('cuda')

        generated_ids = self.dg_model.model.generate(
            input_ids=source_encoding['input_ids'],
            attention_mask=source_encoding['attention_mask'],
            num_beams=generate_count,
            num_return_sequences=generate_count,
            max_length=TARGET_MAX_TOKEN_LEN,
            repetition_penalty=2.5,
            length_penalty=1.0,
            early_stopping=True,
            use_cache=True
        )

        preds = {
            self.tokenizer.decode(generated_id, skip_special_tokens=False, clean_up_tokenization_spaces=True)
            for generated_id in generated_ids
        }

        return ''.join(preds)

    def _correct_index_of(self, text:str, substring: str, start_index: int = 0):
        try:
            index = text.index(substring, start_index)
        except ValueError:
            index = -1

        return index

    def _replace_all_extra_id(self, text: str):
        new_text = text
        start_index_of_extra_id = 0

        while (self._correct_index_of(new_text, '<extra_id_') >= 0):
            start_index_of_extra_id = self._correct_index_of(new_text, '<extra_id_', start_index_of_extra_id)
            end_index_of_extra_id = self._correct_index_of(new_text, '>', start_index_of_extra_id)

            new_text = new_text[:start_index_of_extra_id] + '<sep>' + new_text[end_index_of_extra_id + 1:]

        return new_text

#Sence2vec Generation


In [ ]:
class Sense2VecDistractorGeneration():
    def __init__(self):
        self.s2v = Sense2Vec().from_disk('s2v_old')

    def generate(self, answer: str, desired_count: int) -> List[str]:
        distractors = []
        answer = answer.lower()
        answer = answer.replace(" ", "_")

        sense = self.s2v.get_best_sense(answer)

        if not sense:
            return []

        most_similar = self.s2v.most_similar(sense, n=desired_count)

        for phrase in most_similar:
            normalized_phrase = phrase[0].split("|")[0].replace("_", " ").lower()

            if normalized_phrase.lower() != answer: #TODO: compare the stem of the words (e.g. wrote, writing)
                distractors.append(normalized_phrase.capitalize())

        return list(OrderedDict.fromkeys(distractors))

##Questions

In [ ]:
class Question:
    def __init__(self, answerText:str, questionText: str = '', distractors: List[str] = []):
        self.answerText = answerText
        self.questionText = questionText
        self.distractors = distractors

#MCQ Generation

In [ ]:
class MCQGenerator():
    def __init__(self, is_verbose=False):
        start_time = time.perf_counter()
        print('Loading ML Models...')

        # Currently not used
        self.answer_generator = AnswerGenerator()
        self.answer_generator.ag_model.to('cuda:0')
        print('Loaded AnswerGenerator in', round(time.perf_counter() - start_time, 2), 'seconds.') if is_verbose else ''

        self.question_generator = QuestionGenerator()
        self.question_generator.qg_model.to('cuda:0')
        print('Loaded QuestionGenerator in', round(time.perf_counter() - start_time, 2), 'seconds.') if is_verbose else ''

        self.distractor_generator = DistractorGenerator()
        self.distractor_generator.dg_model.to('cuda:0')
        print('Loaded DistractorGenerator in', round(time.perf_counter() - start_time, 2), 'seconds.') if is_verbose else ''

        self.sense2vec_distractor_generator = Sense2VecDistractorGeneration()
        print('Loaded Sense2VecDistractorGenerator in', round(time.perf_counter() - start_time, 2), 'seconds.') if is_verbose else ''

    # Main function

    def generate_mcq_questions(self, dataset: pd.DataFrame, desired_total_count: int) -> List[Question]:
        topics = dataset['topic'].unique()

        questions = []

        for topic in topics:
            topic_dataset = dataset[dataset['topic'] == topic]
            # Generate questions for the current topic
            topic_questions = self._generate_question_answer_pairs(topic_dataset['context'].iloc[0], desired_count)
            topic_questions = self._generate_distractors(topic_dataset['context'].iloc[0], topic_questions)
            questions.extend(topic_questions)


        # Display the generated questions
            for i, question in enumerate(questions, 1):
                print(f"{i}: {question.questionText}")
                print(f"Answer: {question.answerText}")

                print("Distractors:")
                for distractor in question.distractors:
                    print(distractor)

                print("\n")  # Add a new line between questions

        return questions

    def _generate_answers(self, context: str, desired_count: int) -> List[Question]:
        answers = self.answer_generator.generate(context, desired_count)
        answers = self._generate_multiple_answers_according_to_desired_count(context, desired_count)

        print(answers)
        unique_answers = remove_duplicates(answers)

        questions = []
        for answer in unique_answers:
            questions.append(Question(answer))

        return questions

    def _generate_questions(self, context: str, questions: List[Question]) -> List[Question]:
        for question in questions:
            question.questionText = self.question_generator.generate(question.answerText, context)

        return questions

    def _generate_question_answer_pairs(self, context: str, desired_count: int) -> List[Question]:
        context_splits = self._split_context_according_to_desired_count(context, desired_count)

        questions = []

        for split in context_splits:
            answer, question = self.question_generator.generate_qna(split)
            questions.append(Question(answer.capitalize(), question))

        questions = list(toolz.unique(questions, key=lambda x: x.answerText))

        return questions

    def _generate_distractors(self, context: str, questions: List[Question]) -> List[Question]:
        for question in questions:
            t5_distractors = self.distractor_generator.generate(5, question.answerText, question.questionText, context)
            # distractors = t5_distractors
            if len(t5_distractors) < 3:
                s2v_distractors = self.sense2vec_distractor_generator.generate(question.answerText, 3)
                distractors = t5_distractors + s2v_distractors
            else:
                distractors = t5_distractors

            distractors = remove_duplicates(distractors)
            distractors = remove_distractors_duplicate_with_correct_answer(question.answerText, distractors)

            question.distractors = distractors

        return questions


    # Helper functions
    def _generate_answer_for_each_sentence(self, context: str) -> List[str]:
        sents = sent_tokenize(context)

        answers = []
        for sent in sents:
            answers.append(self.answer_generator.generate(sent, 1)[0])

        return answers

    #TODO: refactor to create better splits closer to the desired amount
    def _split_context_according_to_desired_count(self, context: str, desired_count: int) -> List[str]:
        if desired_count == 0:
            return [context]

        sents = sent_tokenize(context)
        sent_ratio = len(sents) / desired_count

        context_splits = []

        if sent_ratio < 1:
            return sents
        else:
            take_sents_count = int(sent_ratio + 1)

            start_sent_index = 0

            while start_sent_index < len(sents):
                context_split = ' '.join(sents[start_sent_index: start_sent_index + take_sents_count])
                context_splits.append(context_split)
                start_sent_index += take_sents_count - 1

        return context_splits



#Main

In [ ]:
# Check if GPU is available
if torch.cuda.is_available():
    # Move the MCQ_Generator to GPU
    MCQ_Generator = MCQGenerator(True)

    dataset = pd.read_csv('train.csv')
    MCQ_Generator.generate_mcq_questions(dataset, 50)
else:
    print("GPU not available. Please ensure that your system has a compatible GPU.")